In [1]:
import pandas as pd
import os
import csv
import numpy as np
from scipy import stats
from matplotlib import pyplot as plt
%matplotlib
import matplotlib.lines as mlines
import matplotlib.pyplot as plt

Using matplotlib backend: Qt5Agg


In [2]:
from Experimentos2 import Experimentos, labelLines

[Errno 2] No such file or directory: '.\\Data\\5cc50C - max av.csv'
[Errno 2] No such file or directory: '.\\Data\\5cc50C - fspot.csv'
[Errno 2] No such file or directory: '.\\Data\\5cc50C - max av cup.csv'
[Errno 2] No such file or directory: '.\\Data\\5cc50C - frente.csv'
[Errno 2] No such file or directory: '.\\Data\\1cc54C - max av.csv'
[Errno 2] No such file or directory: '.\\Data\\1cc54C - fspot.csv'
[Errno 2] No such file or directory: '.\\Data\\1cc54C - max av cup.csv'
[Errno 2] No such file or directory: '.\\Data\\1cc54C - frente.csv'
[Errno 2] No such file or directory: '.\\Data\\25cc54C - max av.csv'
[Errno 2] No such file or directory: '.\\Data\\25cc54C - fspot.csv'
[Errno 2] No such file or directory: '.\\Data\\25cc54C - max av cup.csv'
[Errno 2] No such file or directory: '.\\Data\\25cc54C - frente.csv'
[Errno 2] No such file or directory: '.\\Data\\10cc60C - max av.csv'
[Errno 2] No such file or directory: '.\\Data\\10cc60C - fspot.csv'
[Errno 2] No such file or director

In [3]:
velos = pd.read_excel('Velocidades y punto.xlsx')

# calcular K, n, Ty para todos los experimentos según temperatura a escoger freely, guardar en alguno de los DF.

In [15]:
for exp in Experimentos:
    #updateo densidad
    exp.param['densidad']=1340
    #calculo cos angulo
    exp.param['slope'] = np.sin(np.radians(exp.param['angulo inclinacion']))

In [18]:
#Añado velos a params x facilidad
for exp in Experimentos:
    for i in range(33):
        if exp.param['NOMBRE'][0]==velos.iloc[i,:]['nombre']:
            exp.param['v1'] = velos.iloc[i,:]['v1']
            exp.param['v2'] = velos.iloc[i,:]['v2']
            exp.param['L*'] = velos.iloc[i,:]['L*']
            exp.param['t*'] = velos.iloc[i,:]['t*']

In [37]:
#Estimo Jeffreys parte primera
for exp in Experimentos:
    
    exp.param['numerador'] = 9.8*exp.param['densidad']*exp.param['result: espesor max final']*exp.param['result: espesor max final']
    
    if exp.param['v1'][0] > 0:
        exp.param['denominador1'] = 3*exp.param['v1']
    if exp.param['v2'][0] > 0:
        exp.param['denominador2'] = 3*exp.param['v2']

In [38]:
#Estimo Jeffreys parte final
for exp in Experimentos:
    if exp.param['v1'][0] > 0:
        exp.param['visc1'] = exp.param['numerador']/exp.param['denominador1']
    if exp.param['v2'][0] > 0:
        exp.param['visc2'] = exp.param['numerador']/exp.param['denominador2']

In [55]:
#Largo según Lister para flujos newtonianos
for exp in Experimentos:
    exp.dinamica['numerador']=exp.param['caudal'][0]*exp.param['caudal'][0]*exp.param['densidad'][0]*9.8*exp.param['slope'][0]
    if exp.param['v1'][0] > 0:
        exp.dinamica['denominador']=exp.param['visc1'][0]*exp.dinamica['avance: ancho maximo']*exp.dinamica['avance: ancho maximo']
        exp.dinamica['division'] = np.cbrt(exp.dinamica['numerador'][0]/exp.dinamica['denominador'])
        exp.dinamica['Lteonew'] = exp.dinamica['division']*exp.dinamica['avance: tiempo']

In [68]:
#se arman GRUPOS SEGUN CAUDAL
from itertools import groupby
count = 0

for key, group in groupby(sorted(listalargonewtoniano, key=lambda x: x['caudal']), key=lambda x: x['caudal']):
    print(key)
    count += 1

1e-06
5e-06
1e-05
1.7e-05
2.5e-05


In [80]:
listalargonewtoniano = []

for exp in Experimentos:
    holder = {} 
    holder['caudal'] = exp.param['caudal'][0]
    holder['angulo'] = exp.param['angulo inclinacion'][0]#en grados
    holder["rango temperatura"] = rango_temperatura(exp.param["t max"][0])

    if exp.param['v1'][0] > 0:
        holder['nombre'] = exp.param['NOMBRE'][0]
        holder['tiempo'] = exp.dinamica['avance: tiempo']
        holder['distancia'] = exp.dinamica['avance: distancia desde punto eyeccion']
        holder['newtoniano'] = exp.dinamica['Lteonew']
        #holder['teyec'] = exp.param['tiempo fin eyeccion']

    listalargonewtoniano.append(holder)

In [76]:
#FUNCION Q DEVUELVE RANGO [1-7] SEGUN TEMP MAXIMA DEL EXP
def rango_temperatura(temp):
    if temp < 40:
        return 1
    if temp < 45:
        return 2
    if temp < 50:
        return 3
    if temp < 55:
        return 4
    if temp < 60:
        return 5
    if temp < 65:
        return 6
    if temp < 70:
        return 7

In [77]:
#GENERA PATCHES DE COLORES SEGUN TEMPERATURA
import matplotlib.patches as mpatches

pat1 = mpatches.Patch(color='C1', label='temp < 40')
pat2 = mpatches.Patch(color='C2', label='temp < 45')
pat3 = mpatches.Patch(color='C3', label='temp < 50')
pat4 = mpatches.Patch(color='C4', label='temp < 55')
pat5 = mpatches.Patch(color='C5', label='temp < 60')
pat6 = mpatches.Patch(color='C6', label='temp < 65')
pat7 = mpatches.Patch(color='C7', label='temp < 70')
pat8 = mpatches.Patch(color='black', label='15° slope')

numerocolores = 7
colors = ['pat{}'.format(i+1) for i in range(numerocolores)] + [pat8]

print(colors)

['pat1', 'pat2', 'pat3', 'pat4', 'pat5', 'pat6', 'pat7', <matplotlib.patches.Patch object at 0x0000026A996C2CC0>]


In [81]:
#plot listalargonewtoniano
fig = plt.figure()
axes = []

#este plot hace cuantos subplot necesite segun un contador que da el numero de
#grupos generados en el groupby
for x in range(count):
    axes.append(fig.add_subplot(2, 3, x + 1))

#UNA COLUMNA DE GRAFICOS CON VARIAS FILAS SEGUN GRUPO
#se generan los ejes de cada plot
#i es el indice de lista y luego cada value en lista (???)
for i, (key, group) in enumerate(groupby(sorted(listalargonewtoniano, key=lambda x: x['caudal']), key=lambda x: x['caudal'])):
    print(i, key)

    for exp in group:
        axes[i].legend([key])
        if exp['angulo'] == 15:
            axes[i].loglog(exp['tiempo'], exp['distancia'], color='C{}'.format(exp['rango temperatura']),
                           linestyle='--')
            axes[i].loglog(exp['tiempo'], exp['newtoniano'], color='C{}'.format(exp['rango temperatura']),
                           linestyle='--') 
        else:
            axes[i].loglog(exp['tiempo'], exp['distancia'], color='C{}'.format(exp['rango temperatura']))
            axes[i].legend([key])
            axes[i].loglog(exp['tiempo'], exp['newtoniano'], color='C{}'.format(exp['rango temperatura']))
            axes[i].legend([key])
        
legend1 = plt.legend(handles=[pat1, pat2, pat3, pat4, pat5, pat6, pat7], bbox_to_anchor=(1.8, 0.5), loc=5)
plt.gca().add_artist(legend1)

linea1= ax.plot([],[], label='15°', linestyle='--', color='black')
linea2= ax.plot([],[], label='10°', color='black')
plt.legend(bbox_to_anchor=(1.8, 0.5), loc=6)
labelLines(plt.gca().get_lines(),align=False,color='k')
fig.show()

0 1e-06


KeyError: 'tiempo'

In [101]:
from scipy.optimize import curve_fit

def func(x, k, c):
    return c*x^k

xData = Experimentos[24].dinamica['avance: tiempo']
yData = Experimentos[24].dinamica['avance: distancia desde punto eyeccion']

popt, pcov = curve_fit(func, xData.dtypes, yData)

TypeError: cannot compare a dtyped [float64] array with a scalar of type [bool]

In [106]:
xData.shape

(48,)

In [99]:
Experimentos[22].dinamica.tail()

,Unnamed: 0,avance: tiempo,avance: distancia desde punto eyeccion,avance: distancia hacia atrás,avance: largo total flujo,avance: ancho maximo,numerador,denominador,division,Lteonew
44,44,6793.333333,0.325,0.073,0.398,0.29,2.393117e-09,450.163533,0.000175,1.185625
45,45,8008.333333,0.335,0.073,0.408,0.29,2.393117e-09,450.163533,0.000175,1.397677
46,46,10966.333333,0.370,0.073,0.443,0.29,2.393117e-09,450.163533,0.000175,1.913930
47,47,23966.333333,0.410,0.073,0.483,0.29,2.393117e-09,450.163533,0.000175,4.182791
48,48,55066.333333,0.415,0.073,0.488,0.29,2.393117e-09,450.163533,0.000175,9.610604


In [84]:
for i, exp in enumerate(Experimentos):
    print(i, exp.param['NOMBRE'])
    
# exp 22 y 24 temperatura ambiente

0 0    5cc50C
Name: NOMBRE, dtype: object
1 0    1cc54C
Name: NOMBRE, dtype: object
2 0    25cc54C
Name: NOMBRE, dtype: object
3 0    10cc60C
Name: NOMBRE, dtype: object
4 0    25cc60C
Name: NOMBRE, dtype: object
5 0    1cc59C
Name: NOMBRE, dtype: object
6 0    1cc47C
Name: NOMBRE, dtype: object
7 0    1cc55C
Name: NOMBRE, dtype: object
8 0    25cc58C
Name: NOMBRE, dtype: object
9 0    25cc70C
Name: NOMBRE, dtype: object
10 0    10cc67C
Name: NOMBRE, dtype: object
11 0    10cc59C
Name: NOMBRE, dtype: object
12 0    5cc63C
Name: NOMBRE, dtype: object
13 0    5cc59C
Name: NOMBRE, dtype: object
14 0    10cc49C
Name: NOMBRE, dtype: object
15 0    25cc62C
Name: NOMBRE, dtype: object
16 0    1cc62C
Name: NOMBRE, dtype: object
17 0    5cc53C
Name: NOMBRE, dtype: object
18 0    5a25cc56C
Name: NOMBRE, dtype: object
19 0    5cc57C
Name: NOMBRE, dtype: object
20 0    25cc50C
Name: NOMBRE, dtype: object
21 0    10cc55C
Name: NOMBRE, dtype: object
22 0    1cc24C
Name: NOMBRE, dtype: object
23 0   

In [45]:
pd.set_option("display.max_columns",33)
Experimentos[6].param.head()

,densidad manjar,modelo 3D,volumen total teorico,t amb,termocupla,caudal,angulo inclinacion,NOMBRE,Obs,Exp #,uso manjar,t max,fecha,min,max,framei,framef,result: largo final,result: largo total,result: ancho max final,result: espesor max final,result: distancia espesor final,densidad,slope,v1,v2,L*,t*,numerador,denominador1,visc1
0,1240,NaN,0.00143,23.9,"si, punto eyeccion",0.000001,10,1cc47C,NaN,7,2,46.7,26/06/2016 0:00,28,52,63,1800,0.29,0.338,0.27,0.021,0.25,1340,0.173648,0.000199,0.0,0.338,1609.6,5.791212,0.000597,9700.522613


In [57]:
Experimentos[6].dinamica.tail()

,Unnamed: 0,avance: tiempo,avance: distancia desde punto eyeccion,avance: distancia hacia atrás,avance: largo total flujo,avance: ancho maximo,numerador,denominador,division,Lteonew
25,25,1196.1,0.25,0.040,0.290,0.250,2.280348e-09,606.282663,0.000156,0.186013
26,26,1269.6,0.26,0.042,0.302,0.254,2.280348e-09,625.838917,0.000154,0.195365
27,27,1363.1,0.27,0.044,0.314,0.260,2.280348e-09,655.755329,0.000152,0.206513
28,28,1465.1,0.28,0.046,0.326,0.264,2.280348e-09,676.087624,0.000150,0.219719
29,29,1609.6,0.29,0.048,0.338,0.270,2.280348e-09,707.168098,0.000148,0.237799
